In [34]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import copy

import torch_geometric.nn as pyg_nn
import torch_geometric.utils as pyg_utils

import time
from datetime import datetime

import networkx as nx
import numpy as np
import torch
import torch.optim as optim

from torch_geometric.datasets import TUDataset
from torch_geometric.datasets import Planetoid
from torch_geometric.data import DataLoader
from torch_geometric.utils import from_networkx

import torch_geometric.transforms as T

#from tensorboardX import SummaryWriter
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt

In [79]:
# output_dim replaced by 1
output_dim = 1
class GNNStack(nn.Module):
    def __init__(self, input_dim, hidden_dim):
        super(GNNStack, self).__init__()
        self.convs = nn.ModuleList()
        self.convs.append(self.build_conv_model(input_dim, hidden_dim))
        for l in range(2):
            self.convs.append(self.build_conv_model(hidden_dim, hidden_dim))

        # post-message-passing
        self.post_mp = nn.Sequential(
            nn.Linear(hidden_dim, hidden_dim), nn.Dropout(0.25), 
            nn.Linear(hidden_dim, output_dim))

        self.dropout = 0.25
        self.num_layers = 3

    def build_conv_model(self, input_dim, hidden_dim):
        # refer to pytorch geometric nn module for different implementation of GNNs.
        return pyg_nn.GCNConv(input_dim, hidden_dim)
    
    def forward(self, data):
        x, edge_index, batch = data.x, data.edge_index, data.batch
        if data.num_node_features == 0:
          x = torch.ones(data.num_nodes, 1)

        for i in range(self.num_layers):
            x = self.convs[i](x, edge_index)
            emb = x
            x = F.relu(x)
            x = F.dropout(x, p=self.dropout, training=self.training)

        x = self.post_mp(x)
        return x

    def loss(self, pred, actual):
        return F.mse_loss(pred, actual)

In [22]:
LG = nx.read_graphml('experiments/preprocessed/san_francisco.graphml')

In [83]:
LG = nx.read_graphml('experiments/preprocessed/san_francisco.graphml')
node_feature_list = ['speed_mph_p85', 'length', 'oneway']
edge_feature_list = ['x','y']

lg_node_attr = {}
node_relabel = {}

IDs = {}
cur_ID = 0

#To make torch_geometric.data extraction easier, remove every node that doesn't have ALL of the features in node_feature_list
remove = []
insert = []
num_train = 24580*0.8
cur_num_train = 0
#seperate into train and test
for key,_ in LG.nodes(data=True):
    new_edge_data = copy.deepcopy(LG.nodes()[key])
    skip = False
    for k in list(new_edge_data.keys()):
        if k not in node_feature_list:
            del new_edge_data[k]
    for k in node_feature_list:
        if k not in new_edge_data:
            skip = True
            break
    if skip == True:
        remove.append(key)
        continue
    x = []
    edge_x_y = {}
    for k,v in new_edge_data.items():
        if k == 'speed_mph_p85':
            edge_x_y['y'] = [v]
            continue
        x.append(v)
    edge_x_y['x'] = x
    if cur_num_train > num_train:
        edge_x_y['train_mask'] = False
        edge_x_y['test_mask'] = True
    else:
        edge_x_y['train_mask'] = True
        edge_x_y['test_mask'] = False 
    lg_node_attr[cur_ID] = edge_x_y
    node_relabel[key] = cur_ID
    IDs[cur_ID] = key
    cur_ID+=1
    cur_num_train+=1
    
for i in remove:
    LG.remove_node(i)
    
nx.relabel.relabel_nodes(LG,node_relabel,False)
print(lg_node_attr[20579])
nx.set_node_attributes(LG,lg_node_attr)
count = 0
for n in LG.nodes(data=True):
    count+=1
print(count)

nx.set_edge_attributes(LG,{})
for n in LG.nodes(data=True):
    print(n)
    break
print('\n')
for e in LG.edges(data=True):
    print(e)
    break

#nx.write_graphml(LG,'experiments/preprocessed/' + CITY_NAME + ".graphml")

pyData = from_networkx(LG)

{'y': [18.855], 'x': [False, 153.292], 'train_mask': False, 'test_mask': True}
20580
(20579, {'oneway': False, 'length': 153.292, 'speed_mph_p85': 18.855, 'y': [18.855], 'x': [False, 153.292], 'train_mask': False, 'test_mask': True})


(20579, 8428, {})


In [86]:
print(pyData.num_node_features)
loader = DataLoader([pyData], batch_size=1)
print(loader.dataset)
print(len(loader.dataset))

2
[Data(edge_index=[2, 60626], length=[20580], oneway=[20580], speed_mph_p85=[20580], test_mask=[20580], train_mask=[20580], x=[20580, 2], y=[20580, 1])]
1


In [85]:
    test_loader = loader = DataLoader([pyData], batch_size=1)

    # build model
    model = GNNStack(max(pyData.num_node_features, 1), 1)
    opt = optim.Adam(model.parameters(), lr=0.01)
    
    # train
    for epoch in range(1000):
        total_loss = 0
        model.train()
        for batch in loader:
            print(batch.train_mask, '----')
            opt.zero_grad()
            pred = model(batch)
            label = batch.y
            pred = pred[batch.train_mask]
            label = label[batch.train_mask]
            loss = model.loss(pred, label)
            loss.backward()
            opt.step()
            total_loss += loss.item() * batch.num_graphs
        total_loss /= len(loader.dataset)
        #writer.add_scalar("loss", total_loss, epoch)

        if epoch % 10 == 0:
            test_acc = test(test_loader, model)
            print("Epoch {}. Loss: {:.4f}. Test accuracy: {:.4f}".format(
                epoch, total_loss, test_acc))
            #writer.add_scalar("test accuracy", test_acc, epoch)

    #return model


tensor([False,  True,  True,  ...,  True,  True,  True]) ----
tensor([[-0.6908],
        [-0.6006],
        [-0.6006],
        ...,
        [-0.6006],
        [-0.6908],
        [-0.6006]], grad_fn=<IndexBackward>)
tensor([[20.7269],
        [15.5725],
        [28.0510],
        ...,
        [18.9080],
        [26.5269],
        [13.1970]])
HI



Epoch 0. Loss: 535.1110. Test accuracy: 0.0000
tensor([False,  True,  True,  ...,  True,  True,  True]) ----
tensor([[-0.5832],
        [-0.5832],
        [-0.5832],
        ...,
        [-0.5832],
        [-0.6808],
        [-0.6808]], grad_fn=<IndexBackward>)
tensor([[20.7269],
        [15.5725],
        [28.0510],
        ...,
        [18.9080],
        [26.5269],
        [13.1970]])
HI



tensor([False,  True,  True,  ...,  True,  True,  True]) ----
tensor([[-0.5654],
        [-0.5654],
        [-0.5654],
        ...,
        [-0.5654],
        [-0.6708],
        [-0.6708]], grad_fn=<IndexBackward>)
tensor([[20.7269],
        [15.5725],
  

tensor([False,  True,  True,  ...,  True,  True,  True]) ----
tensor([[-0.0949],
        [-0.4511],
        [-0.0949],
        ...,
        [-0.4511],
        [-0.4511],
        [-0.0949]], grad_fn=<IndexBackward>)
tensor([[20.7269],
        [15.5725],
        [28.0510],
        ...,
        [18.9080],
        [26.5269],
        [13.1970]])
HI



tensor([False,  True,  True,  ...,  True,  True,  True]) ----
tensor([[-0.0693],
        [-0.0693],
        [-0.0693],
        ...,
        [-0.0693],
        [-0.4411],
        [-0.0693]], grad_fn=<IndexBackward>)
tensor([[20.7269],
        [15.5725],
        [28.0510],
        ...,
        [18.9080],
        [26.5269],
        [13.1970]])
HI



tensor([False,  True,  True,  ...,  True,  True,  True]) ----
tensor([[-0.4312],
        [-0.0433],
        [-0.0433],
        ...,
        [-0.0433],
        [-0.0433],
        [-0.0433]], grad_fn=<IndexBackward>)
tensor([[20.7269],
        [15.5725],
        [28.0510],
        ...,
        [18.9080]

tensor([False,  True,  True,  ...,  True,  True,  True]) ----
tensor([[ 0.6456],
        [ 0.6456],
        [ 0.6456],
        ...,
        [ 0.6456],
        [-0.2133],
        [ 0.6456]], grad_fn=<IndexBackward>)
tensor([[20.7269],
        [15.5725],
        [28.0510],
        ...,
        [18.9080],
        [26.5269],
        [13.1970]])
HI



tensor([False,  True,  True,  ...,  True,  True,  True]) ----
tensor([[ 0.6826],
        [ 0.6826],
        [ 0.6826],
        ...,
        [-0.2034],
        [ 0.6826],
        [ 0.6826]], grad_fn=<IndexBackward>)
tensor([[20.7269],
        [15.5725],
        [28.0510],
        ...,
        [18.9080],
        [26.5269],
        [13.1970]])
HI



tensor([False,  True,  True,  ...,  True,  True,  True]) ----
tensor([[-0.1936],
        [ 0.7202],
        [ 0.7202],
        ...,
        [-0.1936],
        [ 0.7202],
        [ 0.7202]], grad_fn=<IndexBackward>)
tensor([[20.7269],
        [15.5725],
        [28.0510],
        ...,
        [18.9080]

tensor([False,  True,  True,  ...,  True,  True,  True]) ----
tensor([[1.6850],
        [1.6850],
        [1.6850],
        ...,
        [1.6850],
        [1.6850],
        [0.0208]], grad_fn=<IndexBackward>)
tensor([[20.7269],
        [15.5725],
        [28.0510],
        ...,
        [18.9080],
        [26.5269],
        [13.1970]])
HI



tensor([False,  True,  True,  ...,  True,  True,  True]) ----
tensor([[0.0304],
        [1.7353],
        [0.0304],
        ...,
        [1.7353],
        [1.7353],
        [0.0304]], grad_fn=<IndexBackward>)
tensor([[20.7269],
        [15.5725],
        [28.0510],
        ...,
        [18.9080],
        [26.5269],
        [13.1970]])
HI



tensor([False,  True,  True,  ...,  True,  True,  True]) ----
tensor([[1.7861],
        [1.7861],
        [1.7861],
        ...,
        [1.7861],
        [1.7861],
        [1.7861]], grad_fn=<IndexBackward>)
tensor([[20.7269],
        [15.5725],
        [28.0510],
        ...,
        [18.9080],
        [26.5269

tensor([False,  True,  True,  ...,  True,  True,  True]) ----
tensor([[3.0485],
        [3.0485],
        [0.2492],
        ...,
        [3.0485],
        [0.2492],
        [0.2492]], grad_fn=<IndexBackward>)
tensor([[20.7269],
        [15.5725],
        [28.0510],
        ...,
        [18.9080],
        [26.5269],
        [13.1970]])
HI



tensor([False,  True,  True,  ...,  True,  True,  True]) ----
tensor([[3.1123],
        [3.1123],
        [3.1123],
        ...,
        [3.1123],
        [3.1123],
        [3.1123]], grad_fn=<IndexBackward>)
tensor([[20.7269],
        [15.5725],
        [28.0510],
        ...,
        [18.9080],
        [26.5269],
        [13.1970]])
HI



tensor([False,  True,  True,  ...,  True,  True,  True]) ----
tensor([[3.1766],
        [3.1766],
        [3.1766],
        ...,
        [3.1766],
        [3.1766],
        [0.2679]], grad_fn=<IndexBackward>)
tensor([[20.7269],
        [15.5725],
        [28.0510],
        ...,
        [18.9080],
        [26.5269

...

tensor([False,  True,  True,  ...,  True,  True,  True]) ----
tensor([[21.8293],
        [21.8293],
        [21.8293],
        ...,
        [ 4.1617],
        [21.8293],
        [21.8293]], grad_fn=<IndexBackward>)
tensor([[20.7269],
        [15.5725],
        [28.0510],
        ...,
        [18.9080],
        [26.5269],
        [13.1970]])
HI



tensor([False,  True,  True,  ...,  True,  True,  True]) ----
tensor([[ 4.1665],
        [21.8292],
        [21.8292],
        ...,
        [ 4.1665],
        [ 4.1665],
        [21.8292]], grad_fn=<IndexBackward>)
tensor([[20.7269],
        [15.5725],
        [28.0510],
        ...,
        [18.9080],
        [26.5269],
        [13.1970]])
HI



tensor([False,  True,  True,  ...,  True,  True,  True]) ----
tensor([[21.8292],
        [21.8292],
        [21.8292],
        ...,
        [21.8292],
        [21.8292],
        [21.8292]], grad_fn=<IndexBackward>)
tensor([[20.7269],
        [15.5725],
        [28.0510],
        ...,
        [18.9080]

tensor([False,  True,  True,  ...,  True,  True,  True]) ----
tensor([[21.8269],
        [21.8269],
        [21.8269],
        ...,
        [21.8269],
        [21.8269],
        [21.8269]], grad_fn=<IndexBackward>)
tensor([[20.7269],
        [15.5725],
        [28.0510],
        ...,
        [18.9080],
        [26.5269],
        [13.1970]])
HI



tensor([False,  True,  True,  ...,  True,  True,  True]) ----
tensor([[ 4.2827],
        [21.8267],
        [21.8267],
        ...,
        [ 4.2827],
        [ 4.2827],
        [21.8267]], grad_fn=<IndexBackward>)
tensor([[20.7269],
        [15.5725],
        [28.0510],
        ...,
        [18.9080],
        [26.5269],
        [13.1970]])
HI



tensor([False,  True,  True,  ...,  True,  True,  True]) ----
tensor([[ 4.2876],
        [ 4.2876],
        [21.8267],
        ...,
        [21.8267],
        [21.8267],
        [21.8267]], grad_fn=<IndexBackward>)
tensor([[20.7269],
        [15.5725],
        [28.0510],
        ...,
        [18.9080]

tensor([False,  True,  True,  ...,  True,  True,  True]) ----
tensor([[21.8271],
        [21.8271],
        [21.8271],
        ...,
        [21.8271],
        [21.8271],
        [21.8271]], grad_fn=<IndexBackward>)
tensor([[20.7269],
        [15.5725],
        [28.0510],
        ...,
        [18.9080],
        [26.5269],
        [13.1970]])
HI



tensor([False,  True,  True,  ...,  True,  True,  True]) ----
tensor([[21.8271],
        [21.8271],
        [21.8271],
        ...,
        [21.8271],
        [21.8271],
        [21.8271]], grad_fn=<IndexBackward>)
tensor([[20.7269],
        [15.5725],
        [28.0510],
        ...,
        [18.9080],
        [26.5269],
        [13.1970]])
HI



tensor([False,  True,  True,  ...,  True,  True,  True]) ----
tensor([[21.8273],
        [21.8273],
        [21.8273],
        ...,
        [21.8273],
        [ 4.4053],
        [21.8273]], grad_fn=<IndexBackward>)
tensor([[20.7269],
        [15.5725],
        [28.0510],
        ...,
        [18.9080]

tensor([False,  True,  True,  ...,  True,  True,  True]) ----
tensor([[ 4.5134],
        [21.8253],
        [21.8253],
        ...,
        [21.8253],
        [21.8253],
        [21.8253]], grad_fn=<IndexBackward>)
tensor([[20.7269],
        [15.5725],
        [28.0510],
        ...,
        [18.9080],
        [26.5269],
        [13.1970]])
HI



Epoch 960. Loss: 111.0601. Test accuracy: 0.0000
tensor([False,  True,  True,  ...,  True,  True,  True]) ----
tensor([[21.8252],
        [21.8252],
        [21.8252],
        ...,
        [21.8252],
        [ 4.5183],
        [21.8252]], grad_fn=<IndexBackward>)
tensor([[20.7269],
        [15.5725],
        [28.0510],
        ...,
        [18.9080],
        [26.5269],
        [13.1970]])
HI



tensor([False,  True,  True,  ...,  True,  True,  True]) ----
tensor([[ 4.5232],
        [21.8253],
        [21.8253],
        ...,
        [21.8253],
        [21.8253],
        [21.8253]], grad_fn=<IndexBackward>)
tensor([[20.7269],
        [15.5725],


tensor([False,  True,  True,  ...,  True,  True,  True]) ----
tensor([[21.8278],
        [21.8278],
        [21.8278],
        ...,
        [21.8278],
        [21.8278],
        [ 4.6324]], grad_fn=<IndexBackward>)
tensor([[20.7269],
        [15.5725],
        [28.0510],
        ...,
        [18.9080],
        [26.5269],
        [13.1970]])
HI



tensor([False,  True,  True,  ...,  True,  True,  True]) ----
tensor([[21.8278],
        [21.8278],
        [21.8278],
        ...,
        [ 4.6373],
        [21.8278],
        [21.8278]], grad_fn=<IndexBackward>)
tensor([[20.7269],
        [15.5725],
        [28.0510],
        ...,
        [18.9080],
        [26.5269],
        [13.1970]])
HI



tensor([False,  True,  True,  ...,  True,  True,  True]) ----
tensor([[21.8279],
        [ 4.6423],
        [21.8279],
        ...,
        [21.8279],
        [21.8279],
        [21.8279]], grad_fn=<IndexBackward>)
tensor([[20.7269],
        [15.5725],
        [28.0510],
        ...,
        [18.9080]

In [61]:
def test(loader, model, is_validation=False):
    model.eval()

    correct = 0
    for data in loader:
        with torch.no_grad():
            pred = model(data)
            pred = pred.argmax(dim=1)
            label = data.y

        mask = data.val_mask if is_validation else data.test_mask
        # node classification: only evaluate on nodes in test set
        pred = pred[mask]
        label = data.y[mask]
        #print(label, pred)
        correct += pred.eq(label).sum().item()
    total = 0
    for data in loader.dataset:
        total += torch.sum(data.test_mask).item()
    return correct / total